In [1]:
# Librerías.
import pandas as pd
import Funciones as f
import unidecode
from docx import Document
from docx.shared import Inches
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scikit_posthocs as sp

In [2]:
%%capture
%run "35. Comparaciones estadísticas de cambios de tiempo por ítem.ipynb"

In [ ]:
for Nombre_df, df in dfs_Finales.items():

    print("\n" + "="*50)
    print(f"Análisis post hoc de tiempospara el conjunto de datos: {Nombre_df}")
    print("\n" + "="*50)

    # Extraer las variables significativas.
    Variables_Significativas = Todas_Variables_Significativas['Cambios_Tiempo_Por_Item_Significativos'][Nombre_df]

    # Diccionario para almacenar los resultados de Dunn.
    Resultados_Post_Hoc = {}

    # Aplicar prueba de Dunn por cada variable significativa.
    for Variable in Variables_Significativas:

        # Eliminar filas con NaN en la variable específica o en la categoría.
        Datos_Limpios = df[['Categoria_PASO_2023', Variable]].dropna()

        Matriz_Dunn = sp.posthoc_dunn(
            Datos_Limpios,
            val_col = Variable,
            group_col = 'Categoria_PASO_2023',
            p_adjust = 'holm'
        )
        Resultados_Post_Hoc[Variable] = Matriz_Dunn
    
    # Inicializar lista de resúmenes interpretativos.
    Resumen_Significancias = []

    # Recorrer cada variable significativa.
    for Nombre_Variable, Matriz in Resultados_Post_Hoc.items():
        Comparaciones_Significativas = []

        for Categoria_1 in Matriz.index:
            for Categoria_2 in Matriz.columns:
                if Categoria_1 < Categoria_2:  # Evita duplicados.
                    P_Valor = Matriz.loc[Categoria_1, Categoria_2]
                    if P_Valor < 0.05:
                        Comparaciones_Significativas.append(
                            f"- {Categoria_1} vs {Categoria_2} (p = {P_Valor:.4f})"
                        )

        if Comparaciones_Significativas:
            Resumen = (
                f"\n📊 Comparaciones significativas para **{Nombre_Variable}**:\n" +
                "\n".join(Comparaciones_Significativas)
            )
        else:
            Resumen = (
                f"\n📊 No se encontraron diferencias significativas "
                f"entre categorías para **{Nombre_Variable}**."
            )

        Resumen_Significancias.append(Resumen)

    # Imprimir todos los resúmenes.
    for R in Resumen_Significancias:
        print(R)
